In [1]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import requests
import time
import random

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST driver version for 96.0.4664
Driver [C:\Users\m_che\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


In [5]:
# Job titles loading
job_titles_df = pd.read_csv('../resources/job_titles.csv')
job_titles_df

,job_titles
0,Data Analyst
1,Data Engineer
2,Data Scientist
3,Data Architect
4,Big Data Software Engineer
5,Machine Learning Engineer
6,Business Intelligence Analyst
7,Logistics Analyst
8,Business Systems Analyst
9,Marketing Analyst


In [6]:
#Dataframe used to store job postings
column_names = ["Job Title", "Company Name", "Company Location","Salary", "Description", "Link", "Job Category"]
job_listings_df = pd.DataFrame(columns = column_names)

job_titles = []
company_names = []
company_locations = []
job_salaries = []
job_descriptions = []
job_links = []
job_categories = []

In [7]:
for job_category in job_titles_df['job_titles']:
    baseUrl = 'https://www.usajobs.gov/Search/Results?jt=' + job_category + '&p='
    page_counter = 1
    while True:
        url = baseUrl + str(page_counter)
        page_counter += 1
        print(url)
        browser.visit(url)
        time.sleep(2)
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        try:
            jobs = soup.find_all('div', 'usajobs-search-result--core')
        except AttributeError:
            break

        for job in jobs:
            job_title = job.a.text.strip()  
            job_href = job.a.get('href')
            job_department = job.find('h5', 'usajobs-search-result--core__department').text.strip()
            try:
                job_location = job.find('h4','usajobs-search-result--core__location').span.a.get('data-name')
            except AttributeError:
                job_location = 'No location provided'

            try:
                job_salary = job.find('div','usajobs-search-result--core__details').ul.li.text.strip()
            except AttributeError:
                job_salary = 'No salary provided'


            url_job_description = "https://www.usajobs.gov" + job_href
            response = requests.get(url_job_description)
            description_soup = BeautifulSoup(response.text, 'html.parser')
            try:
                job_description = description_soup.find('div',{'id':'duties'}).text.strip() + description_soup.find('div',{'id':'requirements'}).text.strip() 
            except AttributeError:
                job_description = 'No description provided'    

            print(job_category)
            print(job_title)
#             print(job_href)
#             print(job_department)
#             print(job_location)
#             print(job_salary)
    #         print(job_description)
            print("---------------")
            job_titles.append(job_title)
            company_names.append(job_department)
            company_locations.append(job_location)
            job_salaries.append(job_salary)
            job_descriptions.append(job_description)
            job_links.append(url_job_description)
            job_categories.append(job_category)

        next_page = soup.find('a','usajobs-search-pagination__next-page')
        if next_page.get('aria-hidden')=="true":
            break
            

job_listings_df["Job Title"]=job_titles
job_listings_df["Company Name"]=company_names
job_listings_df["Company Location"]=company_locations
job_listings_df["Salary"]=job_salaries
job_listings_df["Description"]=job_descriptions
job_listings_df["Link"]=job_links
job_listings_df["Job Category"]=job_categories

job_listings_df.to_csv('mw_govjob_listings.csv', index=False)
    

https://www.usajobs.gov/Search/Results?jt=Data Analyst&p=1
Data Analyst
Data Analyst
---------------
Data Analyst
IT Specialist (DATAMGT) (Data and Reporting Analyst)
---------------
Data Analyst
Program Analyst (Data Science)
---------------
Data Analyst
Health System Specialist (Data Analyst)
---------------
Data Analyst
Interdisciplinary (Data Research Analyst)
---------------
Data Analyst
Supervisory Program Analyst (Data Scientist)
---------------
Data Analyst
Supervisory Program Analyst (Data Scientist)
---------------
Data Analyst
FINANCIAL MANAGEMENT ANALYST (DATA ANALYTICS)
---------------
Data Analyst
Data Scientist
---------------
Data Analyst
Data Scientist
---------------
Data Analyst
Data Scientist
---------------
Data Analyst
Supervisory Performance Improvement / Data Analyst
---------------
Data Analyst
Supervisory Performance Improvement / Data Analyst
---------------
Data Analyst
Data Technician NF2
---------------
Data Analyst
Supervisory Management and Program Analy

KeyboardInterrupt: 